In [3]:
import inspect
from collections import OrderedDict
from inspect import Parameter
from functools import wraps
from typing import *

from mousse import validate, parse

In [11]:
def validate_args_type(func: Callable, /, *args, **kwargs) -> Union[bool, str]:
    signature = inspect.signature(func)
    parameters = signature.parameters
    
    parameters = OrderedDict(parameters.items())
    score = 0
    kwargs = kwargs.copy()
    
    has_var_args = False
    for param in parameters.values():
        if param.kind == param.VAR_POSITIONAL:
            has_var_args = True
            break
            
    if has_var_args:
        parameters.pop(param.name)
        
    has_var_kwargs = False
    for param in parameters.values():
        if param.kind == param.VAR_KEYWORD:
            has_var_kwargs = True
            break
            
    if has_var_kwargs:
        parameters.pop(param.name)

    positional_only_params = [
        param for param in parameters.values()
        if param.kind == param.POSITIONAL_ONLY
    ]

    if len(positional_only_params) > len(args):
        return False, "Missing positional only params"
    
    for arg, param in zip(args, positional_only_params):
        parameters.pop(param.name)
    
        if param.annotation in (inspect._empty, Any):
            continue

        if not validate(param.annotation, arg):
            return False, f"Wrong type for {param.name}. Expect type {param.annotation.__name__}"
        
    args = args[len(positional_only_params):]
    
    positional_or_keyword_params = [
        param for param in parameters.values()
        if param.kind == param.POSITIONAL_OR_KEYWORD
    ]
    
    if len(args) > len(positional_or_keyword_params):
        if not has_var_args:
            return False, "Too many arguments"
    
    for arg, param in zip(args, positional_or_keyword_params):
        parameters.pop(param.name)
        if param.annotation in (inspect._empty, Any):
            continue

        if not validate(param.annotation, arg):
            return False, f"Wrong type for {param.name}. Expect type {param.annotation.__name__}"

        
    for key in kwargs:
        if key not in parameters:
            if not has_var_kwargs:
                return False, f"Unknown keyword argument: {key}"
    
    for key, val in kwargs.items():
        if key in parameters:
            param = parameters.pop(key)
            if param.annotation in (inspect._empty, Any):
                continue
                
            if not validate(param.annotation, val):
                return False, f"Wrong type for {param.name}. Expect type {param.annotation.__name__}"
        
    for param in parameters.values():
        if param.default == inspect._empty:
            return False, f"Missing value for {param.name}"
    
    return True, ""

In [20]:
def foo(name: str, age: int, school: str, address: str = None, **kwargs):
    print(name, age, school, address)
    
@wraps(foo)
def bar(*args, **kwargs):
    return foo(*args, **kwargs)

In [21]:
validate_args_type(foo, "Dat", 19, "TL", degree="Msc")

(True, '')

In [22]:
def validate_signature(func: Callable = None, param_annotation: bool = True, return_annotation: bool = True):
    def decorator(func: Callable):        
        signature = inspect.signature(func)

        @wraps(func)
        def wrapper(*args, **kwargs):
            if param_annotation:
                valid, err = validate_args_type(func, *args, **kwargs)
                assert valid, err

            output = func(*args, **kwargs)
            if return_annotation and signature.return_annotation is not inspect._empty:
                assert validate(signature.return_annotation, output), \
                    f"Incorrect return type: {type(output)}. Correct return type: {signature.return_annotation}"
        
            return output
        
        return wrapper
    
    if func is not None:
        return decorator(func)
    
    return decorator

In [23]:
@validate_signature
def foo(name: str, age: int, school: str, address: str = None, **kwargs):
    print(name, age, school, address)

In [25]:
foo(12, 18, "HUST")

AssertionError: Wrong type for name. Expect type str

In [83]:
param = signature.parameters["name"]
param.annotation

inspect._empty

In [66]:
foo('a', 1, school=None, address={})

a 1 None {}


In [69]:
Parameter.POSITIONAL_ONLY

TypeError: foo() missing 3 required positional arguments: 'name', 'age', and 'school'

In [76]:
a = (1, 2, 3, 4, 5)
a = iter(a)

In [77]:
next(a)

1

In [78]:
list(a)

[2, 3, 4, 5]